# Error Analysis on 8760 profiles (22 profiles)

In [1]:
#importing packages needed for analysis
import os
import numpy as np
import pandas as pd
import math
from pandas import DataFrame

path = os.getcwd()
#print(path)

#this code creates an output directory in the parent director, if one does not exist yet
#Note: this is where all of the output files will be written, since outputs are large this saves space in git
path = os.getcwd()
parent = os.path.dirname(path)
outputs_dir = parent+'\outputs\error_analysis'
if not os.path.exists(outputs_dir):
    os.makedirs(outputs_dir)
print('output files are written out in parent directory: '+outputs_dir)

##UNCOMMENT WHICH PROFILE BEING ANALYZED 
x = 'load'
x2 = 'Load'

#x = 'solar'
#x2 = 'Solar_Gen'

#x = 'wind'
#x2 = 'Wind_Gen'

profiles = ['norm','timeofday','bind_1hr','seq_2hr','seq_4hr','seq_8hr','seq_12hr','seq_24hr','seq_48hr',
           'seq_120hr','bind_8760hr','1daytype_monthly_24hr','1daytype_season_24hr','2daytype_monthly_24hr',
            '2daytype_season_24hr','2daytype_month_4hr','2daytype_bimonth_4hr','2daytype_season_4hr',
            '3daytype_monthly_24hr','3daytype_season_24hr','3daytype_annual_24hr','3daytype_bimonth_4hr',
            '3daytype_seasonbased_4hr','3daytype_season_4hr']

output files are written out in parent directory: C:\Users\tgoforth\Documents\IPM temporal resolution project\outputs\error_analysis


In [2]:
# create DFs for error analysis
# DF to use for regional error analysis, like RMSE and number of representative hours 
regions = pd.read_csv('../outputs/load_long_format.csv')
regions = regions['Region'].unique()
regions = pd.DataFrame(regions).rename(columns={0:'Region'})
#print(regions)

# DF to use for RMSE and Diff for entire 
profile_df = pd.DataFrame(profiles).rename(columns={0:'Profile'})
#print(profile_df)

## General Error Analysis

difference in values 
root mean square for each region
other analyses? 

## Loop and create error files 

In [5]:
rep_hours = regions
reg_RMSE = regions
prof_RMSE = []

for i in range(len(profiles)):
    segments = pd.read_csv('../outputs/'+x+'/'+x+'_segments_'+profiles[i]+'.csv')
    hours = pd.read_csv('../outputs/'+x+'/'+x+'_8760_'+profiles[i]+'.csv')
    
    # use groupby region to count number of representative hours in each region
    aggregations = {'Avg':'count'}
    stat1 = segments.groupby(['Region'],as_index=False).agg(aggregations)
    stat1 = stat1.rename(columns={'Avg': profiles[i]})
    
    rep_hours = pd.merge(rep_hours, stat1, on='Region', how='left')
    
    # calculate RMSE 
    stat2 = hours[['Region','R_Group','Hour_Tot',x2,'Avg']].copy()
    
    # export differences to CSV
    stat2['Diff'] = stat2[x2] - stat2['Avg'] 
    stat2.to_csv('../outputs/error_analysis/'+x+'_'+profiles[i]+'_diff.csv')
    
    # regional RMSE
    stat3 = stat2.copy()
    stat3['Square'] = stat3['Diff']**2
    stat_reg = stat3.groupby('Region',as_index=False).agg({'Square' : sum})
    stat_reg = stat_reg.rename(columns={'Square':'Square_Sum'})
    stat_reg['Mean'] = stat_reg['Square_Sum'] / 8760 
    stat_reg[profiles[i]] = stat_reg['Mean']**(1/2)
    stat_reg = stat_reg.drop(columns={'Square_Sum','Mean'})

    reg_RMSE = pd.merge(reg_RMSE, stat_reg, on='Region', how='left')
    
    # profile RMSE
    stat4 = stat2.copy()
    stat4['Square'] = stat2['Diff']**2
    stat4 = stat4.agg({'Square' : sum})
    #stat4 = stat4.rename(columns={'Square':'Square_Sum'})
    stat4 = stat4 / 551880 
    stat4[profiles[i]] = stat4**(1/2)

    prof_RMSE.append(profiles[i]+':'+str(stat4['Square']))
    
print(prof_RMSE)
rep_hours.to_csv('../outputs/error_analysis/rep_hours_summary.csv')
reg_RMSE.to_csv('../outputs/error_analysis/regional_RMSE.csv')

['norm:236857.65845438399', 'timeofday:236857.65845438399', 'bind_1hr:4655639.08915639', 'seq_2hr:53974.30452453432', 'seq_4hr:236023.08347919837', 'seq_8hr:907178.5618757701', 'seq_12hr:1901401.225221063', 'seq_24hr:2270296.34221887', 'seq_48hr:2362869.6775981644', 'seq_120hr:2616097.925640297', 'bind_8760hr:0.0', '1daytype_monthly_24hr:901030.7250626988', '1daytype_season_24hr:1133232.2366984615', '1daytype_season_24hr:1133232.2366984615', '2daytype_monthly_24hr:857489.1810810082', '2daytype_season_24hr:1057414.0568355091', '2daytype_month_4hr:236023.08347919837', '2daytype_bimonth_4hr:236023.08347919837', '2daytype_season_4hr:1731379.141900342', '3daytype_monthly_24hr:745670.2386578114', '3daytype_season_24hr:1291570.5151967073', '3daytype_annual_24hr:2859031.0465331795', '3daytype_bimonth_4hr:1295909.0563590021', '3daytype_seasonbased_4hr:1188247.302622916', '3daytype_season_4hr:1630927.7227763452']


## Create outputs: total number of representative hours, RMS

In [6]:
# calculate RMSE 
stat2 = x_hours[['Region','R_Group','Hour_Tot',x2,'Avg']].copy()
stat2 = pd.merge(stat2,stat1,on='Region',how='left')
stat2 = stat2[['Region','R_Group','Rep_Count','Hour_Tot',x2,'Avg']]

#print(stat2)

stat2['Diff'] = stat2[x2] - stat2['Avg'] 
stat2['Square'] = stat2['Diff']**2
stat_reg = stat2.groupby(['Region','Avg','Hour_Tot'],as_index=False).agg({'Square' : sum})
stat_reg = stat_reg.rename(columns = {'Square':'Square_Sum'}) 
stat_reg['Mean'] = stat_reg['Square_Sum'] / stat_reg['Hour_Tot'] 
stat_reg['RMSE'] = stat_reg['Mean']**(1/2)
stat_reg = stat_reg.drop(columns={'Square_Sum','Mean'})
print(stat_reg)

stat3 = pd.merge(stat2,stat_reg,on=['Region','Avg','Hour_Tot'],how='left')
print(stat3)

stat3.to_csv('../outputs/error_analysis/'+x+'_'+x_name+'_error.csv')
stat_reg.to_csv('../outputs/error_analysis/'+x+'_'+x_name+'region_RMSE.csv')

        Region           Avg  Hour_Tot         RMSE
0     ERC_REST  29443.558559       111  1914.297419
1     ERC_REST  29796.642659       361  2119.770006
2     ERC_REST  32400.546218       119  3739.593398
3     ERC_REST  32597.629139       151  3285.439791
4     ERC_REST  33159.873362       229  3955.101522
...        ...           ...       ...          ...
3397  WEC_SDGE   3093.250000        16   285.685907
3398  WEC_SDGE   3145.583333        12   197.046297
3399  WEC_SDGE   3184.750000        20   442.881460
3400  WEC_SDGE   3582.200000        20   513.011462
3401  WEC_SDGE   3701.300000        20   510.829531

[3402 rows x 4 columns]
          Region R_Group  Rep_Count  Hour_Tot   Load           Avg  \
0       ERC_REST     ERC         54       111  26989  29443.558559   
1       ERC_REST     ERC         54       361  27006  29796.642659   
2       ERC_REST     ERC         54       361  27025  29796.642659   
3       ERC_REST     ERC         54       361  27059  29796.642659   
4